# XX Soluciones — Clases 12 a 15

Guiones de referencia con ejemplos funcionales y comentarios breves.

## Clase 12: conexión desde Python a MySQL (driver + consulta + DataFrame)

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text

# Credenciales y parámetros de conexión
DB_USER = "usuario"
DB_PASSWORD = "clave"
DB_HOST = "localhost"
DB_PORT = 3306
DB_NAME = "tienda"

# Driver sugerido: mysql+mysqlconnector o mysql+pymysql
engine = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# Consulta básica y carga a DataFrame
query = text("SELECT id, nombre, email, fecha_alta FROM clientes LIMIT 100;")
with engine.connect() as connection:
    result = connection.execute(query)
    df_clientes = pd.DataFrame(result.fetchall(), columns=result.keys())

df_clientes.head()

## Clase 13: limpieza/transformación con pandas para preparación

In [ ]:
# Normalizar columnas y tipos
df_clientes = df_clientes.rename(columns={"fecha_alta": "fecha_alta"})
df_clientes["fecha_alta"] = pd.to_datetime(df_clientes["fecha_alta"], errors="coerce")

# Limpiar texto y duplicados
df_clientes["email"] = df_clientes["email"].str.lower().str.strip()
df_clientes["nombre"] = df_clientes["nombre"].str.title()
df_clientes = df_clientes.drop_duplicates(subset=["id"])

# Completar nulos y crear nueva columna
df_clientes = df_clientes.fillna({"email": "sin_email@ejemplo.com"})
df_clientes["anio_alta"] = df_clientes["fecha_alta"].dt.year

df_clientes.head()

## Clase 14: pipeline ETL básico (extraer, transformar, cargar)

In [ ]:
# Extraer
def extraer_clientes(engine):
    consulta = "SELECT id, nombre, email, fecha_alta FROM clientes;"
    return pd.read_sql(consulta, engine)


# Transformar
def transformar_clientes(df):
    df = df.copy()
    df["email"] = df["email"].str.lower().str.strip()
    df["fecha_alta"] = pd.to_datetime(df["fecha_alta"], errors="coerce")
    df = df.drop_duplicates(subset=["id"])
    df["anio_alta"] = df["fecha_alta"].dt.year
    return df


# Cargar
def cargar_clientes(df, output_path, engine, table_name):
    df.to_csv(output_path, index=False)
    df.to_sql(table_name, con=engine, if_exists="replace", index=False)


df_raw = extraer_clientes(engine)
df_limpio = transformar_clientes(df_raw)
cargar_clientes(df_limpio, "clientes_limpios.csv", engine, "clientes_limpios")

df_limpio.head()

## Clase 15: fuente externa + avance de proyecto final (checkpoint)

In [ ]:
import requests

# Fuente externa (API)
url = "https://jsonplaceholder.typicode.com/users"
response = requests.get(url, timeout=10)
response.raise_for_status()
df_api = pd.DataFrame(response.json())

df_api.head()

In [ ]:
# Checkpoint de proyecto final
checkpoint = {
    "tablas_origen": ["clientes", "ventas"],
    "fuente_externa": url,
    "transformaciones": ["limpieza de emails", "normalización de fechas", "nuevas columnas"],
    "proximo_paso": "definir métricas y modelo de datos para el dashboard"
}

pd.Series(checkpoint)